In [15]:
import pandas as pd

# Load the main dataset
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')

X.set_index('Primary ID', inplace=True)
y.set_index('Primary ID', inplace=True)

# Load the column names from the Excel files (assuming they are in the first row)
padel_cols = pd.read_excel('Padel_cols.xlsx', header=None).iloc[0].dropna().tolist()
spartan_cols = pd.read_excel('Spartan_cols.xlsx', header=None).iloc[0].dropna().tolist()
swissadme_cols = pd.read_excel('Swissadme_cols.xlsx', header=None).iloc[0].dropna().tolist()


Check non-numeric data

In [16]:
non_numeric_columns = X.select_dtypes(exclude=['number']).columns
non_numeric_columns

Index([], dtype='object')

Check rows with nans

In [17]:
def check_nans(data):
    rows_with_nans = data.isnull().any(axis=1)
    num_rows_with_nans = rows_with_nans.sum()
    total_rows = len(data)
    fraction_rows_with_nans = num_rows_with_nans / total_rows

    print(f"Number of rows with NaNs: {num_rows_with_nans}")
    print(f"Fraction of rows with NaNs: {fraction_rows_with_nans:.2f}")

    # Identify columns with NaNs
    columns_with_nans = data.columns[data.isnull().any()].tolist()
    num_columns_with_nans = len(columns_with_nans)

    print(f"Columns with NaNs: {columns_with_nans}")

    # Print detailed information about NaNs in each column
    nan_info = data.isnull().sum()
    print("\nDetailed NaN information:")
    print(nan_info[nan_info > 0])
    print("Number of columns with nan values:")
    print(f"{num_columns_with_nans} columns out of the total {len(data.columns)} columns")

In [18]:
from sklearn.decomposition import PCA
import numpy as np
import joblib

def perform_pca(X_subset, subset_name, explained_variance_threshold=0.95):
    # Print data nans before standardization
    print("NANs before standardization")
    check_nans(X_subset)

    # Drop zero variance columns
    zero_variance_columns = X_subset.loc[:, X_subset.std() == 0].columns
    X_subset = X_subset.drop(columns=zero_variance_columns)
    # save the columns to drop
    joblib.dump(zero_variance_columns, subset_name + '_cols_to_drop.pkl')
    
    # Standardize the data if necessary
    mean = X_subset.mean()
    std = X_subset.std()
    X_standardized = (X_subset - mean) / std
    # save the values for standardization
    joblib.dump(mean, subset_name + '_mean.pkl')
    joblib.dump(std, subset_name + '_std.pkl')

    # Print data nans before standardization
    print("NANs after standardization")
    check_nans(X_standardized)

    # Initialize PCA
    pca = PCA()

    # Fit PCA
    pca.fit(X_standardized)

    # Calculate cumulative explained variance
    cum_var_explained = np.cumsum(pca.explained_variance_ratio_)
    
    # Determine the number of components needed to reach the explained variance threshold
    num_components = np.argmax(cum_var_explained >= explained_variance_threshold) + 1
    
    # Apply PCA with the selected number of components
    pca = PCA(n_components=num_components)
    X_pca = pca.fit_transform(X_standardized)

    # Save the scaler
    joblib.dump(pca, subset_name + '_pca_fit.pkl')
    
    return X_pca, num_components, pca.explained_variance_ratio_

# Perform PCA on each set of columns
X_padel_pca, padel_n_components, padel_variance_ratio = perform_pca(X[padel_cols], subset_name='padel')
X_spartan_pca, spartan_n_components, spartan_variance_ratio = perform_pca(X[spartan_cols], subset_name='spartan')
X_swissadme_pca, swissadme_n_components, swissadme_variance_ratio = perform_pca(X[swissadme_cols], subset_name='swissadme')

# Display the number of components retained
print(f"Padel: {padel_n_components} components retained")
print(f"Spartan: {spartan_n_components} components retained")
print(f"SwissADME: {swissadme_n_components} components retained")


NANs before standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 1444 columns
NANs after standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 1079 columns
NANs before standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 23 columns
NANs after standardization
Number of rows with NaNs: 0
Fraction of rows with NaNs: 0.00
Columns with NaNs: []

Detailed NaN information:
Series([], dtype: int64)
Number of columns with nan values:
0 columns out of the total 23 columns
NANs before standardization
Number of rows with NaNs: 

Remove NaN rows

After performing PCA, merge the datasets back together

In [19]:
# Convert PCA results to DataFrames with 'Primary ID' as the index
X_padel_pca_df = pd.DataFrame(X_padel_pca, index=X.index)
X_spartan_pca_df = pd.DataFrame(X_spartan_pca, index=X.index)
X_swissadme_pca_df = pd.DataFrame(X_swissadme_pca, index=X.index)

# Ensure that the other features DataFrame is also indexed by 'Primary ID'
other_features_cols = ["API %", "Plast %", "ST-Diam (mm)", "3PBT-Diam (mm)", "3PBT-Radius (mm)"]
X_other_features = X[other_features_cols].set_index(X.index)
print(f"Other features: {X_other_features}")

# Normalize the rest of the columns
mean = X_other_features.mean()
std = X_other_features.std()
X_standardized = (X_other_features - mean) / std
# save the values for standardization
joblib.dump(mean, 'non_pca_features_mean.pkl')
joblib.dump(std, 'non_pca_features_std.pkl')

# Merge the PCA-transformed data back together with the rest of the features
X_final = pd.concat([X_padel_pca_df, X_spartan_pca_df, X_swissadme_pca_df, X_standardized], axis=1)

# Make sure all the column names are strings
X_final.columns = X_final.columns.astype(str)

# Verify the final DataFrame shape and columns
print(f"Final DataFrame shape: {X_final.shape}")
print(X_final)

Other features:             API %  Plast %  ST-Diam (mm)  3PBT-Diam (mm)  3PBT-Radius (mm)
Primary ID                                                                
BCS1_S1         5      0.0          1.99            1.88             0.940
BCS1_S2         5      0.0          1.91            1.77             0.885
BCS1_S3         5      0.0          1.91            1.82             0.910
BCS1_S4         5      0.0          1.91            1.91             0.955
BCS1_S6        15      0.0          1.81            1.61             0.805
...           ...      ...           ...             ...               ...
PS-S16         25     15.0          1.78            1.86             0.930
PS-S17         25     15.0          1.86            1.86             0.930
PS-S18         25     15.0          1.92            1.83             0.915
PS-S19         25     15.0          1.84            1.89             0.945
PS-S20         25     15.0          1.90            1.77             0.885

[578 row

Check for duplicates in the column names between the 3 data sources (Padel, Spartan, SwissADME)

In [20]:
X_final.index

Index(['BCS1_S1', 'BCS1_S2', 'BCS1_S3', 'BCS1_S4', 'BCS1_S6', 'BCS1_S7',
       'BCS1_S8', 'BCS1_S9', 'BCS1_S10', 'BCS1_S11',
       ...
       'PS-S11', 'PS-S12', 'PS-S13', 'PS-S14', 'PS-S15', 'PS-S16', 'PS-S17',
       'PS-S18', 'PS-S19', 'PS-S20'],
      dtype='object', name='Primary ID', length=578)

In [21]:
X_final.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '0', '1', '2', '3', '4',
       '5', '6', '0', '1', '2', '3', '4', '5', '6', 'API %', 'Plast %',
       'ST-Diam (mm)', '3PBT-Diam (mm)', '3PBT-Radius (mm)'],
      dtype='object')

In [22]:
y.index

Index(['BCS1_S1', 'BCS1_S2', 'BCS1_S3', 'BCS1_S4', 'BCS1_S6', 'BCS1_S7',
       'BCS1_S8', 'BCS1_S9', 'BCS1_S10', 'BCS1_S11',
       ...
       'PS-S11', 'PS-S12', 'PS-S13', 'PS-S14', 'PS-S15', 'PS-S16', 'PS-S17',
       'PS-S18', 'PS-S19', 'PS-S20'],
      dtype='object', name='Primary ID', length=578)

In [23]:
y.columns

Index(['ST-Hardness (g)', 'ST-Rigidity at 2% deformation (g)',
       'ST-Rigidity at 4% deformation (g)', 'ST-Peak stress (N/mp)',
       '3PBT-Hardness (g)', '3PBT-Deformation at hardness (mm)',
       '3PBT-Total work (mJ)', '3PBT-Maximum force (N)',
       '3PBT-Peak stress (N/mp)',
       '3PBT-Flexural stress (g/mmp) (Samaro 2021 Prasad 2019)',
       '3PBT-Flexural strain (%)', '3PBT-Breaking distance (mm)',
       '3PBT-Stiffness (N/mm) (Hu 2022)'],
      dtype='object')

In [24]:
X_final.to_csv("X_PCA.csv")
y.to_csv("y_PCA.csv")

In [25]:
X_final

,0,1,2,3,4,5,6,7,8,0,...,2,3,4,5,6,API %,Plast %,ST-Diam (mm),3PBT-Diam (mm),3PBT-Radius (mm)
Primary ID,,,,,,,,,,,,,,,,,,,,,
BCS1_S1,15.138992,4.600423,-12.529111,0.900570,-9.212201,1.494760,3.400860,-0.286404,7.718222,3.071754,...,1.584529,0.467193,0.428449,-0.759455,0.288681,-1.346813,-1.393608,-0.047417,0.184688,0.184688
BCS1_S2,15.138992,4.600423,-12.529111,0.900570,-9.212201,1.494760,3.400860,-0.286404,7.718222,3.071754,...,1.584529,0.467193,0.428449,-0.759455,0.288681,-1.346813,-1.393608,-0.055464,-0.510955,-0.510955
BCS1_S3,15.138992,4.600423,-12.529111,0.900570,-9.212201,1.494760,3.400860,-0.286404,7.718222,3.071754,...,1.584529,0.467193,0.428449,-0.759455,0.288681,-1.346813,-1.393608,-0.055464,-0.194754,-0.194754
BCS1_S4,15.138992,4.600423,-12.529111,0.900570,-9.212201,1.494760,3.400860,-0.286404,7.718222,3.071754,...,1.584529,0.467193,0.428449,-0.759455,0.288681,-1.346813,-1.393608,-0.055464,0.374408,0.374408
BCS1_S6,15.138992,4.600423,-12.529111,0.900570,-9.212201,1.494760,3.400860,-0.286404,7.718222,3.071754,...,1.584529,0.467193,0.428449,-0.759455,0.288681,0.002334,-1.393608,-0.065522,-1.522798,-1.522798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PS-S16,-4.349452,1.014761,-2.473316,-7.683201,2.616664,-13.192463,-1.222705,-7.417322,4.684139,-0.586709,...,-0.856440,-4.459253,-1.663018,3.826590,1.670400,1.351482,1.327694,-0.068540,0.058207,0.058207
PS-S17,-4.349452,1.014761,-2.473316,-7.683201,2.616664,-13.192463,-1.222705,-7.417322,4.684139,-0.586709,...,-0.856440,-4.459253,-1.663018,3.826590,1.670400,1.351482,1.327694,-0.060493,0.058207,0.058207
PS-S18,-4.349452,1.014761,-2.473316,-7.683201,2.616664,-13.192463,-1.222705,-7.417322,4.684139,-0.586709,...,-0.856440,-4.459253,-1.663018,3.826590,1.670400,1.351482,1.327694,-0.054458,-0.131513,-0.131513


# Train Random Forest Multiregressor

In [26]:
# test-train split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

# Initialize the base model
base_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Create a multi-output regressor
multi_output_model = MultiOutputRegressor(base_model)

# Train the model
multi_output_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

Evaluate the model

In [28]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on the test set
y_pred = multi_output_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='variance_weighted')

# Print MSE and target variable names
for target_name, mse_value in zip(y_test.columns, mse):
    print(f"Target: {target_name}, Mean Squared Error: {mse_value}")

print(f"\nOverall R^2 Score: {r2}")


Target: ST-Hardness (g), Mean Squared Error: 3976131.3427370694
Target: ST-Rigidity at 2% deformation (g), Mean Squared Error: 468297.59696120693
Target: ST-Rigidity at 4% deformation (g), Mean Squared Error: 4063831.1736206897
Target: ST-Peak stress (N/mp), Mean Squared Error: 52689873388161.22
Target: 3PBT-Hardness (g), Mean Squared Error: 52320.22397219828
Target: 3PBT-Deformation at hardness (mm), Mean Squared Error: 0.5531760889655172
Target: 3PBT-Total work (mJ), Mean Squared Error: 437.15394373094813
Target: 3PBT-Maximum force (N), Mean Squared Error: 5.014058823622052
Target: 3PBT-Peak stress (N/mp), Mean Squared Error: 551936903803.4458
Target: 3PBT-Flexural stress (g/mmp) (Samaro 2021 Prasad 2019), Mean Squared Error: 363.4307340432587
Target: 3PBT-Flexural strain (%), Mean Squared Error: 23.542600031365815
Target: 3PBT-Breaking distance (mm), Mean Squared Error: 0.5531760889655172
Target: 3PBT-Stiffness (N/mm) (Hu 2022), Mean Squared Error: 2.1177751975224584

Overall R^2 Sc

In [29]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Predict on the test set
y_pred = multi_output_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')  # MSE for each target
r2_per_target = r2_score(y_test, y_pred, multioutput='raw_values')  # R² for each target
r2_overall = r2_score(y_test, y_pred, multioutput='variance_weighted')  # Overall weighted R²

# Calculate variance or range of each target for relative performance
variance_targets = np.var(y_test, axis=0)  # Variance of each target in test set
range_targets = np.ptp(y_test, axis=0)  # Range (max-min) of each target in test set

# Print MSE, R² score, variance, and range for each target
for target_name, mse_value, r2_value, variance_value, range_value in zip(y_test.columns, mse, r2_per_target, variance_targets, range_targets):
    print(f"Target: {target_name}, Mean Squared Error: {mse_value}, R² Score: {r2_value}")
    print(f"Target: {target_name}, Variance: {variance_value}, Range: {range_value}")
    print(f"Relative MSE (MSE/Variance): {mse_value/variance_value if variance_value != 0 else 'Undefined'}")
    print(f"Relative MSE (MSE/Range): {mse_value/range_value if range_value != 0 else 'Undefined'}\n")

# Print overall R² score
print(f"\nOverall R^2 Score: {r2_overall}")


Target: ST-Hardness (g), Mean Squared Error: 3976131.3427370694, R² Score: 0.5402719584211833
Target: ST-Hardness (g), Variance: 8648877.125445899, Range: 12675.0
Relative MSE (MSE/Variance): 0.45972804157881675
Relative MSE (MSE/Range): 313.698725265252

Target: ST-Rigidity at 2% deformation (g), Mean Squared Error: 468297.59696120693, R² Score: 0.4090343329811429
Target: ST-Rigidity at 2% deformation (g), Variance: 792427.7552764567, Range: 3900.0
Relative MSE (MSE/Variance): 0.5909656670188571
Relative MSE (MSE/Range): 120.07630691312998

Target: ST-Rigidity at 4% deformation (g), Mean Squared Error: 4063831.1736206897, R² Score: 0.2051211281517935
Target: ST-Rigidity at 4% deformation (g), Variance: 5112516.28084126, Range: 9550.0
Relative MSE (MSE/Variance): 0.7948788718482066
Relative MSE (MSE/Range): 425.5320600649937

Target: ST-Peak stress (N/mp), Mean Squared Error: 52689873388161.22, R² Score: 0.9859661324027957
Target: ST-Peak stress (N/mp), Variance: 3754479869730116.0, Ra

In [30]:
import joblib

# Save the model
joblib.dump(multi_output_model, 'multi_output_model.pkl')

['multi_output_model.pkl']